<a href="https://colab.research.google.com/github/alexandre-matias1/SRT-filter-data/blob/main/Dados_Sorter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
import plotly.express as px
import pandas as pd
import sqlite3

conn = sqlite3.connect("/content/drive/My Drive/Logs/logs.db")

cursor = conn.cursor()



df = pd.read_excel('xandao 15.12.xlsx', usecols=['Data','Mensagem', 'Tipo'])

############# Criação da tabela de logs #############
# dfLogs = df[df.apply(lambda row: 'NDIR' in row.values, axis=1)]
# dfLogs.to_sql('logs', conn, if_exists='append', index=False)


############# Criação da tabela de recirculação #############
dfRec = df[df.apply(lambda row: 'ACKN' in row.values, axis=1)]
dfRec['Mensagem'] = dfRec['Mensagem'].str.slice(96, 99)
dfRec.drop(columns=['Tipo'], inplace=True)
dfRec['Data'] = dfRec['Data'].str.slice(0, 19)
dfRec['Data'] = pd.to_datetime(dfRec['Data'], format='%d/%m/%Y %H:%M:%S')

dfRec = dfRec.loc[dfRec['Mensagem'] == '999']
dfRec.set_index('Data', inplace=True)
timing = '60T'
dfRec_resample = dfRec.resample(timing).apply(len)
dfRec_resample = dfRec_resample.drop_duplicates()
dfRec_resample.to_sql('recirculacao', conn, if_exists='append', index=True)



############# Criação da tabela de inducao #############
dfInd = df[df.apply(lambda row: 'NDIR' in row.values, axis=1)]
dfInd['Mensagem'] = dfInd['Mensagem'].str.slice(78, 83)
dfInd = dfInd[dfInd['Mensagem'].isin(['IND1A','IND2A','IND1B'])]
dfInd.drop(columns=['Tipo'], inplace=True)
dfInd['Data'] = pd.to_datetime(dfInd['Data'], format='%d/%m/%Y %H:%M:%S.%f')

dfInd_resample = dfInd.groupby(pd.Grouper(key='Data', freq='60T')).count()
total = dfInd_resample['Mensagem'].sum()
dfInd_resample = dfInd_resample.drop_duplicates()
dfInd_resample.to_sql('inducao', conn, if_exists='append', index=True)



############# Criação da tabela de inducao #############

dfRej = df[df.apply(lambda row: 'ACKN' in row.values, axis=1)]
dfRej['Mensagem'] = dfRej['Mensagem'].str.slice(96, 99)
dfRej.drop(columns=['Tipo'], inplace=True)
dfRej['Data'] = dfRej['Data'].str.slice(0, 19)
dfRej['Data'] = pd.to_datetime(dfRej['Data'], format='%d/%m/%Y %H:%M:%S')


dfRejA = dfRej.loc[dfRej['Mensagem'] == 'A09']
dfRejB = dfRej.loc[dfRej['Mensagem'] == 'B14']
dfRejA.set_index('Data', inplace=True)
dfRejB.set_index('Data', inplace=True)


dfRejA_resample = dfRejA.resample(timing).apply(len)
dfRejB_resample = dfRejB.resample(timing).apply(len)
dfRejA_resample = dfRejA_resample.drop_duplicates()
dfRejB_resample = dfRejB_resample.drop_duplicates()

dfRejA_resample['Rejeito A'] = dfRejA_resample
dfRejA_resample['Rejeito B'] = dfRejB_resample
dfRej = dfRejA_resample
dfRej = dfRej.drop(dfRej.columns[0], axis=1)

dfRej.to_sql('rejeito', conn, if_exists='append', index=True)




# ######## Criação da tabela de ociosidade #############

# timer = '0 days 00:02:00'
# timing = '30T'

# dfOci = df[df.apply(lambda row: 'NDIR' in row.values, axis=1)]
# dfOci['Mensagem'] = dfOci['Mensagem'].str.slice(78, 83)
# dfOci['Data'] = dfOci['Data'].str.slice(0,19)
# dfOci = dfOci.loc[dfOci['Mensagem'] == 'IND1A']
# dfOci = dfOci.sort_values(by='Data')
# dfOci = dfOci.reset_index(drop=True)
# dfOci['Data'] = pd.to_datetime(dfOci['Data'], format='%d/%m/%Y %H:%M:%S',errors='coerce')
# dfOci['Diferença'] = dfOci['Data'].diff()
# dfOci = dfOci[dfOci['Diferença'] > pd.Timedelta(timer)]
# dfOci.loc[dfOci['Diferença'] > pd.Timedelta('1:00:00'), 'Diferença'] = pd.Timedelta('0:00:00')
# dfOci.set_index('Data', inplace=True)
# dfOci= dfOci.resample(timing).sum()
# dfOci = dfOci.drop(dfOci.columns[[0,1]], axis=1)
# display(dfOci)

# tempoOcioso = dfOci['Diferença'].sum()
# stringTempo = tempoOcioso
# time_delta = pd.to_timedelta(stringTempo)
# tempoOciosoEmMinutos = time_delta.total_seconds()/60
# print(tempoOciosoEmMinutos)




# figRec = px.line(dfRec_resample, x=dfRec_resample.index, y='Mensagem',
#               title='Quantidade de volumes recirculados no dia (Intervalo de 30 minutos)',
#               labels={'index': 'Hora', 'Praça': 'Qtd Volumes'})

# figRec.update_xaxes(
#     tickformat='%H:%M',
#     tickangle=-45
# )

# figRec.show()
# figRec.write_image("rec.png")





# figInd = px.line(dfInd_resample, x=dfInd_resample.index, y='Mensagem',
#               title='Quantidade de volumes recirculados no dia (Intervalo de 30 minutos)',
#               labels={'index': 'Hora', 'Praça': 'Qtd Volumes'})

# figInd.update_xaxes(
#     tickformat='%H:%M',
#     tickangle=-45
# )

# figInd.show()
# figInd.write_image("ind.png")




<ipython-input-3-baa97000f680>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfRec['Mensagem'] = dfRec['Mensagem'].str.slice(96, 99)
<ipython-input-3-baa97000f680>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfRec.drop(columns=['Tipo'], inplace=True)
<ipython-input-3-baa97000f680>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

4

In [ ]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 2.3 MB/s eta 0:00:00


In [ ]:
import sqlite3
import pandas as pd


conn = sqlite3.connect("/content/drive/My Drive/Logs/logs.db")


recirculacao = "SELECT * FROM recirculacao WHERE Data > '2024-12-15' ORDER BY Data ASC"
inducao = "SELECT * FROM inducao WHERE Data > '2024-12-15' ORDER BY Data ASC"
rejeito = "SELECT * FROM rejeito WHERE Data > '2024-12-15' ORDER BY Data ASC"
resultado_recirculacao = pd.read_sql(recirculacao, conn)
resultado_inducao = pd.read_sql(inducao, conn)
resultado_rejeito = pd.read_sql(rejeito, conn)

resultado_recirculacao = resultado_recirculacao.drop_duplicates()
resultado_inducao = resultado_inducao.drop_duplicates()
resultado_rejeito = resultado_rejeito.drop_duplicates()


display(resultado_inducao)

with pd.ExcelWriter('Dados Sorter.xlsx', engine='xlsxwriter') as writer:
    resultado_recirculacao.to_excel(writer, sheet_name='Recirculação', index=False)
    resultado_inducao.to_excel(writer, sheet_name='Indução', index=True)
    resultado_rejeito.to_excel(writer, sheet_name='Rejeito', index=True)



,Data,Mensagem
0,2024-12-15 05:00:00,20
1,2024-12-15 06:00:00,8
2,2024-12-15 07:00:00,903
3,2024-12-15 08:00:00,1796
4,2024-12-15 09:00:00,2039
5,2024-12-15 10:00:00,1629
6,2024-12-15 11:00:00,960
